In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import datetime

In [2]:
def read_var(varname, filename):
    with open(filename, 'r') as f:
        data = f.readlines()

    for (i,x) in enumerate(data):
        if ' ' + varname in x: start = i
                
    end = i
    for (i,x) in enumerate(data[start:]):
        if not x.strip(): lenblock = i; break
    var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
    return var

In [3]:
def to_xr(var):
    var.to_xarray()

In [4]:
def filesanddates():    
    filename = "/Users/tobias/files.txt"
    with open(filename, 'r') as f:
        data = f.readlines()
    
    years = [int(d[0:4]) for d in data]
    months = [(d[5:7]) for d in data]
    monthss = ['DJ','JF','FM','MA','AM','MJ','JJ','JA','AS','SO','ON','ND']
    mon = [monthss.index(m) for m in months]
    test = [d[0:7] for d in data]
    date = [datetime.datetime(years[i], mon[i]+1, 1) for i in range(len(years))]
    
# filename = '/Users/tobias/Desktop/QBO Ancistrus/test10a.dat'
#    years = range(2002,2013)
#    year = [str(y) for y in range(2002,2013)]
#    months = ['DJ','JF','FM','MA','AM','MJ','JJ','JA','AS','SO','ON','ND']
#    month = [str(m) for m in months]
#    test = []
#    date = []
#    for y in year:
#        for m in month:
#            test.append( y + '_' + m )
#            date.append(datetime.datetime(years[year.index(y)], month.index(m)+1, 1))
#    
#    test = np.delete(test, [0,1,2,3,4,5,6,27,28,29,30,31,32,33,34,35,-8,-7,-6,-5,-4,-3,-2,-1])
#    date = np.delete(date, [0,1,2,3,4,5,6,27,28,29,30,31,32,33,34,35,-8,-7,-6,-5,-4,-3,-2,-1])
    return(test, date)

In [5]:
def readvars(filename):
    varname = 'v_phi'
    var = read_var(varname, filename)

    varname = 'v_z'
    varz = read_var(varname, filename)

    varname = 'model altitude'
    alt = read_var(varname, filename)

    varname = 'model latitude'
    lat = read_var(varname, filename)

    latlist = lat.T.to_xarray().to_array().values.ravel()
    latlist = latlist[~np.isnan(latlist)]
    #print(latlist)
    altlist = alt.T.to_xarray().to_array().values.ravel()
    altlist = altlist[~np.isnan(altlist)]
    #print(altlist)
    varlist = var.to_xarray().to_array().values.ravel()
    varlist.resize(len(altlist),len(latlist))

    varzlist = varz.to_xarray().to_array().values.ravel()
    varzlist.resize(len(altlist),len(latlist))

    #print(np.shape(varlist),np.shape(var))
    
    varlist = varlist[np.newaxis,...]
    varzlist = varzlist[np.newaxis,...]
    return(varlist,varzlist,altlist,latlist)

In [6]:
def readvars2(filename):
    varname = 'v_phi'
    var = read_var(varname, filename)

    varname = 'v_z'
    varz = read_var(varname, filename)

    varname = 'model altitude'
    alt = read_var(varname, filename)

    varname = 'model latitude'
    lat = read_var(varname, filename)

    latlist = lat.T.to_xarray().to_array().values.ravel()
    latlist = latlist[~np.isnan(latlist)]
    #print(latlist)
    altlist = alt.T.to_xarray().to_array().values.ravel()
    altlist = altlist[~np.isnan(altlist)]
    #print(altlist)
    varlist = var.to_xarray().to_array().values.ravel()
    varlist.resize(len(latlist),len(altlist))

    varzlist = varz.to_xarray().to_array().values.ravel()
    varzlist.resize(len(latlist),len(altlist))
    #print(np.shape(varlist),np.shape(var))
    
    varlist = varlist[np.newaxis,...]
    varzlist = varzlist[np.newaxis,...]
    return(varlist,varzlist,altlist,latlist)

In [7]:
#df = pd.DataFrame({'year': [2014],
#...                    'month': [9],
#...                    'day': [6]})
#time = pd.to_datetime(df)

In [8]:
def mkds(varlist,varzlist,latlist, altlist,time):# define data with variable attributes
    data_vars = {'v_phi':(['time', 'alt', 'lat'], varlist, 
                         {'units': 'm/s', 
                          'long_name':'meridional velocity'}),
                 'v_z'  :(['time', 'alt', 'lat'], varzlist, 
                         {'units': 'm/s', 
                          'long_name':'vertical velocity'})}

# define coordinates
    coords = {'time': (['time'], [time]),
         'lat': (['lat'], latlist),
         'alt': (['alt'], altlist)}

# define global attributes
    attrs = {'creation_date':str(datetime.datetime.now()), 
         'author':'Tobias Kerzenmacher', 
         'email':'kerzenmacher@kit.edu'}

# create dataset
    ds = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)
    return(ds)

In [9]:
def mkds2(varlist,varzlist,latlist, altlist,time):# define data with variable attributes
    data_vars = {'v_phi':(['time', 'lat', 'alt'], varlist, 
                         {'units': 'm/s', 
                          'long_name':'meridional velocity'}),
                 'v_z'  :(['time', 'lat', 'alt'], varzlist, 
                         {'units': 'm/s', 
                          'long_name':'vertical velocity'})}

# define coordinates
    coords = {'time': (['time'], [time]),
         'lat': (['lat'], latlist),
         'alt': (['alt'], altlist)}

# define global attributes
    attrs = {'creation_date':str(datetime.datetime.now()), 
         'author':'Tobias Kerzenmacher', 
         'email':'kerzenmacher@kit.edu'}

# create dataset
    ds = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)
    return(ds)

In [14]:
ds2.to_netcdf('est2.nc')

In [ ]:
import datetime

d = datetime.datetime(2019, 4, 13)
print(d)

In [ ]:
for t in test:
    print(t)
    print(date[test.index(t)])

In [11]:
test, date = filesanddates()
#test=test[0:2]
#date=date[0:2]
for i,t in enumerate(test):
    filename = '/Users/tobias/Mount/Ancistrus/test10a_'+t+'/output/test10a.dat'
    time = date[i]
    print(filename,time,i)
    varlist, varzlist, altlist, latlist = readvars(filename)
    ds1 = mkds(varlist,varzlist,latlist, altlist,time)
    if i==0 : 
        ds = mkds(varlist,varzlist,latlist, altlist,time)
    else:
        ds= xr.concat([ds,mkds(varlist,varzlist,latlist, altlist,time)],dim='time')
    print(ds)


/Users/tobias/Mount/Ancistrus/test10a_2002_AS/output/test10a.dat 2002-09-01 00:00:00 0


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze =

<xarray.Dataset>
Dimensions:  (time: 1, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... 0.2047 0.05528
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... -0.002155
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu
/Users/tobias/Mount/Ancistrus/test10a_2002_JA/output/test10a.dat 2002-08-01 00:00:00 1


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze =

<xarray.Dataset>
Dimensions:  (time: 2, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01 2002-08-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... -0.02466 0.03492
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... -0.001071
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu
/Users/tobias/Mount/Ancistrus/test10a_2002_ND/output/test10a.dat 2002-12-01 00:00:00 2


...

/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze =

<xarray.Dataset>
Dimensions:  (time: 97, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01 2002-08-01 ... 2012-02-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... 0.4467 0.1309
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... -0.003378
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu
/Users/tobias/Mount/Ancistrus/test10a_2012_MA/output/test10a.dat 2012-04-01 00:00:00 97


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze =

<xarray.Dataset>
Dimensions:  (time: 98, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01 2002-08-01 ... 2012-04-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... -0.09487 -0.1467
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... 0.001867
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)


In [13]:
test, date = filesanddates()
#test=test[0:2]
#date=date[0:2]
for i,t in enumerate(test):
    filename = '/Users/tobias/Mount/Ancistrus/test10a_'+t+'/output/test10a.dat'
    time = date[i]
    print(filename,time,i)
    varlist, varzlist, altlist, latlist = readvars2(filename)
    #ds1 = mkds2(varlist,varzlist,latlist, altlist,time)
    if i==0 : 
        ds2 = mkds2(varlist,varzlist,latlist, altlist,time)
    else:
        ds2= xr.concat([ds,mkds2(varlist,varzlist,latlist, altlist,time)],dim='time')
    print(ds)


/Users/tobias/Mount/Ancistrus/test10a_2002_AS/output/test10a.dat 2002-09-01 00:00:00 0


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze =

<xarray.Dataset>
Dimensions:  (time: 98, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01 2002-08-01 ... 2012-04-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... -0.09487 -0.1467
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... 0.001867
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu
/Users/tobias/Mount/Ancistrus/test10a_2002_JA/output/test10a.dat 2002-08-01 00:00:00 1


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze =

<xarray.Dataset>
Dimensions:  (time: 98, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01 2002-08-01 ... 2012-04-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... -0.09487 -0.1467
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... 0.001867
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu
/Users/tobias/Mount/Ancistrus/test10a_2002_ND/output/test10a.dat 2002-12-01 00:00:00 2


...

/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze =

<xarray.Dataset>
Dimensions:  (time: 98, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01 2002-08-01 ... 2012-04-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... -0.09487 -0.1467
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... 0.001867
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu
/Users/tobias/Mount/Ancistrus/test10a_2012_MA/output/test10a.dat 2012-04-01 00:00:00 97


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)


<xarray.Dataset>
Dimensions:  (time: 98, alt: 21, lat: 30)
Coordinates:
  * time     (time) datetime64[ns] 2002-09-01 2002-08-01 ... 2012-04-01
  * lat      (lat) float64 -87.0 -81.0 -75.0 -69.0 -63.0 ... 69.0 75.0 81.0 87.0
  * alt      (alt) float64 6.0 9.0 12.0 15.0 18.0 ... 54.0 57.0 60.0 63.0 66.0
Data variables:
    v_phi    (time, alt, lat) float64 -0.01124 -0.1158 0.0 ... -0.09487 -0.1467
    v_z      (time, alt, lat) float64 -4.153e-05 -5.15e-05 ... 0.001867
Attributes:
    creation_date:  2022-04-11 21:09:47.859060
    author:         Tobias Kerzenmacher
    email:          kerzenmacher@kit.edu


/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
/var/folders/00/hbv3fx115yx7drr245f5hxmm0000gn/T/ipykernel_2663/1485784974.py:11: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)


In [ ]:
ds

In [ ]:
ds.v_z[2].plot()

In [ ]:
xr.plot.contour(ds.v_z[2])

In [ ]:
xr.plot.contour(ds2.v_z[2])